Get original DataFrame

In [ ]:
import sys
import plotly.graph_objects as go
import pandas as pd

import numpy as np
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import StandardScaler

sys.path.insert(0, os.path.abspath('../..'))

from python_scripts.api_calls import fetch_item_to_df, fetch_items, get_cookie_from_blob

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

dailyCookie = get_cookie_from_blob()
items = fetch_items()

current_item = fetch_item_to_df(items[4], dailyCookie)
#print(items[4])

#print(current_item.tail())
#print(non_aggregated_item.tail())

df = current_item

os. getcwd()

In [ ]:
# Fill missing values if any
df = df.fillna(method='ffill')

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
df['scaled_price'] = scaler.fit_transform(df[['price_usd']])

In [ ]:
seq_length = 1  # Length of the sequence

# Create columns for each step in the sequence
for i in range(1, seq_length + 1):
    df[f't-{i}'] = df['scaled_price'].shift(i)

# Drop rows with NaN values that were introduced by shifting
df.dropna(inplace=True)

In [ ]:
X_labels = [f't-{i}' for i in range(1, seq_length + 1)]
y_label = 'scaled_price'

X = df[X_labels]
y = df[y_label]

split_train = int(0.7 * len(X))  # First 70% for training
split_test = int(0.9 * len(X))   # Next 20% for testing, leaving last 10% for validation

# Split the data
X_train = X[:split_train]
y_train = y[:split_train]

X_test = X[split_train:split_test]
y_test = y[split_train:split_test]

X_val = X[split_test:]
y_val = y[split_test:]

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))


Creates and fit the LSTM network

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(seq_length, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=3, batch_size=1, verbose=2)


Makes predictions, inverts predictions, and calculate root mean sqaured error

In [ ]:
#trainPredict = model.predict(trainX)
testPredict = model.predict(X_test)

# invert predictions
#trainPredict = scaler.inverse_transform(trainPredict)
unscaledtrainY = scaler.inverse_transform([y_train])
unscaledtestPredict = scaler.inverse_transform(testPredict)
unscaledtestY = scaler.inverse_transform([y_test])
rowunscaledtestY = unscaledtestY.reshape(-1,1)
# calculate root mean squared error
#trainScore = np.sqrt(mean_squared_error(scaledtrainY[0], trainPredict[:,0]))

#print(unscaledtestY, unscaledtestPredict)

#print(testPredict)
#print(testY)
#print(unscaledtestY)

Metric calculation

In [ ]:
testScore = np.sqrt(mean_squared_error(unscaledtestY[0], unscaledtestPredict[:,0]))  #Root mean squared error
r2 = r2_score(unscaledtestY[0], unscaledtestPredict[:, 0]) #R2 score - This provides an indication of the goodness of fit and therefore a measure of how well unseen samples are likely to be predicted by the model. It is the proportion of the variance in the dependent variable that is predictable from the independent variables.
explained_variance = explained_variance_score(unscaledtestY[0], unscaledtestPredict[:, 0])  #Explained variance score  this measures the proportion to which a mathematical model accounts for the variation (dispersion) of a given data set. It is the proportion of the variance in the dependent variable that is predictable from the independent variables
medae = median_absolute_error(unscaledtestY[0], unscaledtestPredict[:, 0])
mae = mean_absolute_error(unscaledtestY[0], unscaledtestPredict[:, 0])
mape = np.mean(np.abs((unscaledtestY[0] - unscaledtestPredict[:, 0]) / unscaledtestY[0])) * 100

print('Test Score: %.2f RMSE' % (testScore))
print('R2 Score: %.2f' % (r2))
print('Explained Variance: %.2f' % (explained_variance))
print('Median Absolute Error: %.2f' % (medae))
print('Mean Absolute Error: %.2f' % (mae))
print('Mean Absolute Percentage Error: %.2f' % (mape)+'%')

Shifts predicitons and plots graphs

In [ ]:
# # shift train predictions for plotting
# trainPredictPlot = np.empty_like(dataset)

# trainPredictPlot[:, :] = np.nan

# #trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

plt.grid()
plt.title("LSTM - Predicted vs Actual")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.plot(unscaledtestPredict , label = "Predicted") #ȳ
plt.plot(rowunscaledtestY, label = "Actual") #ground truth values
plt.legend(loc="upper left")
plt.show()
print(testScore)